In [1]:
from re import search
import nbimporter, random, statistics, time
import numpy as np
from IASearch import Problem, AStar
import jdc

## Un TP taquin

Dans ce TP, nous allons tester l'algorithme $A^*$ avec différentes heuristiques sur le problème du taquin.

Nous allons représenter un état du taquin comme une matrice de taille $n\times n$ (avec $n$ un paramètre du problème) dans laquelle les cellules contiennent les symboles '1', '2', $\ldots$, '$n\times n - 1$', représentant les tuiles du taquin, et le symbole 'e' représentant l'espace vide.

Ainsi, l'état final pour le taquin avec $n=3$ sera donné par la matrice:

| | | |
|-----|-----|-----|
| '1' | '2' | '3' |
| '4' | '5' | '6' |
| '7' | '8' | 'e' |

Commençons par la classe encodant un état de notre problème de recherche.

In [9]:
from re import match
import re
from unittest import case


class SBState():
    
    def __init__(self,n):
        self.n = n
        self.mat = np.array([str(i+1) for i in range(n*n-1)]+['e'])
        self.mat = np.resize(self.mat,(n,n))
        self.final = np.array(self.mat)
        self.pos_e = (self.n-1,self.n-1)
    
    def __str__(self):
        return str(self.mat)+"\n\n"
    
    def __lt__(self,other):
        return str(self.mat) < str(other.mat)
    
    def __hash__(self):
        return hash(str(self.mat))
    
    def __eq__(self,other):
        return np.array_equal(self.mat,other.mat)
    
    def copy(self):
        copy = SBState(self.n)
        copy.mat = np.array(self.mat)
        copy.pos_e = self.pos_e
        return copy

    def swap(self, x,y):
        tmp = self.mat[x]
        self.mat[x] = self.mat[y]
        self.mat[y] = tmp
    
    def actions(self):
        if self.pos_e == (0,0):
            return {'D','R'}
        elif self.pos_e == (0,self.n-1):
            return {'D','L'}
        elif self.pos_e == (self.n-1,0):
            return {'U','R'}
        elif self.pos_e == (self.n-1,self.n-1):
            return {'U','L'}
        elif self.pos_e[0] == 0:
            return {'D','R','L'}
        elif self.pos_e[0] == self.n-1:
            return {'U','R','L'}
        elif self.pos_e[1] == 0:
            return {'U','D','R'}
        elif self.pos_e[1] == self.n-1:
            return {'U','D','L'}
        
        return {'U','D','R','L'}

    def transition(self, action):
        if action == 'U':
            upElt = (self.pos_e[0]-1,self.pos_e[1])
            self.swap(self.pos_e,upElt)
            self.pos_e = upElt
        elif action == 'D':
            downElt = (self.pos_e[0]+1,self.pos_e[1])
            self.swap(self.pos_e,downElt)
            self.pos_e = downElt
        elif action == 'R':
            rightElt = (self.pos_e[0],self.pos_e[1]+1)
            self.swap(self.pos_e,rightElt)
            self.pos_e = rightElt
        elif action == 'L':
            leftElt = (self.pos_e[0],self.pos_e[1]-1)
            self.swap(self.pos_e,leftElt)
            self.pos_e = leftElt
        
        return self.copy()

    def isFinal(self):
        return np.array_equal(self.mat,self.final)
        
    def shuffle(self,t):
        for i in range(t):
            action = random.choice(list(self.actions()))
            self.transition(action)
        return self

À vous de compléter la classe SBState avec les méthodes suivantes :
- actions(self) : retourne l'ensemble des actions possibles dans l'état. C'est un sous-ensemble de $\{$'U','D','R','L'$\}$ où 'U' (resp. 'D', 'R', 'L') correspond au fait de faire bouger l'espace vide vers le haut (resp. le bas, la droite, la gauche). 
- transition(self, action) : retourne le nouvel état obtenu en effectuant 'action' dans l'état courant.
- isFinal(self) : retourne True si l'état est final et False sinon.
- shuffle(self,t) : sur t itérations, cette méthode effectue une action aléatoire et retourne l'état obtenu après les t modifications.

Voici maintenant la classe correspondant au problème du Taquin. 

In [10]:
class SlidingBlock(Problem):
    
    def __init__(self, n, s):
        self.n=n
        self.p_shuffle = s
        self.initial_state = SBState(n).shuffle(self.p_shuffle) 
    
    def actions(self, state):
        return state.actions()
            
    def transition(self, state, action):
        return state.transition(action)
        
    def actionCost(self, state, action):
        return 1
    
    def isFinal(self, state):
        return state.isFinal()
    
    def getInitialState(self):
        return self.initial_state

Testons désormais l'algorithme UniformCostSearch ($A^*$ sans heuristique) sur notre problème. Changer les paramètres $n$ et $s$ pour voir l'influence sur le temps de la recherche. 

In [14]:
#un test simple
from tabnanny import verbose


n = 3 
s = 7
sb = SlidingBlock(n,s)
astar = AStar(sb,lambda x:0)
print(astar.solve())

def evaluate(sb_size,p_shuffle,nb_test,heuristic = lambda x:0):
    times = []
    nbNodes = []
    for i in range(nb_test):
        sb = SlidingBlock(sb_size,p_shuffle)
        astar = AStar(sb,heuristic)
        start = time.time()
        astar.solve()
        end = time.time()
        times.append(end - start)
        nbNodes.append(astar.nbNodes)
    return statistics.mean(times), statistics.mean(nbNodes)

#une mesure de temps
print(evaluate(n,s,20))

0[['1' 'e' '3']
 ['4' '2' '6']
 ['7' '5' '8']]


-------

1[['1' '2' '3']
 ['4' 'e' '6']
 ['7' '5' '8']]

 1[['1' 'e' '3']
 ['4' '2' '6']
 ['7' '5' '8']]

 1[['1' '3' 'e']
 ['4' '2' '6']
 ['7' '5' '8']]


-------

1[['1' '3' 'e']
 ['4' '2' '6']
 ['7' '5' '8']]

 1[['1' 'e' '3']
 ['4' '2' '6']
 ['7' '5' '8']]


-------

1[['1' 'e' '3']
 ['4' '2' '6']
 ['7' '5' '8']]


-------

2[['1' '3' 'e']
 ['4' '2' '6']
 ['7' '5' '8']]

 2[['1' 'e' '3']
 ['4' '2' '6']
 ['7' '5' '8']]


-------

2[['1' 'e' '3']
 ['4' '2' '6']
 ['7' '5' '8']]


-------

3[['1' '3' 'e']
 ['4' '2' '6']
 ['7' '5' '8']]

 3[['1' 'e' '3']
 ['4' '2' '6']
 ['7' '5' '8']]


-------

3[['1' 'e' '3']
 ['4' '2' '6']
 ['7' '5' '8']]


-------

4[['1' '3' 'e']
 ['4' '2' '6']
 ['7' '5' '8']]

 4[['1' 'e' '3']
 ['4' '2' '6']
 ['7' '5' '8']]


-------

4[['1' 'e' '3']
 ['4' '2' '6']
 ['7' '5' '8']]


-------

5[['1' '3' 'e']
 ['4' '2' '6']
 ['7' '5' '8']]

 5[['1' 'e' '3']
 ['4' '2' '6']
 ['7' '5' '8']]


-------

5[['1' 'e' '3']
 ['

KeyboardInterrupt: 

Nous allons maintenant améliorer notre algorithme en rajoutant des heuristiques. 

Coder dans la classe SBState les heuristiques suivantes:
- misplaced(self): compte le nombre de tuiles mal placées.
- manhattan(self): calcule la somme des distances de Manhattan de chaque tuile à sa position dans l'état final. 

In [ ]:
%%add_to SBState      
def misplaced(self):
    #TODO
    
def manhattan(self):
    #TODO

In [ ]:
%%add_to SlidingBlock 
def misplaced(self, state):
    return state.misplaced()
    
def manhattan(self, state):
    return state.manhattan()

Testons maintenant l'algorithme $A^*$ avec ces deux heuristiques.

In [ ]:
#une mesure de temps
print(evaluate(n,s,20, sb.misplaced))
print("\n\n")
print(evaluate(n,s,20, sb.manhattan))

Considérons une relaxation du taquin où une tuile peut bouger d'une case $A$ à une case $B$ si $B$ est vide. 
Le score optimal de cette relaxation définit l'heuristique de Gaschnig.
- L'heuristique de Gaschnig est elle admissible ?
- Comment peut-on calculer l'heuristique de Gaschnig ?
- L'heuristique de Gaschnig domine-t-elle l'heuristique misplaced ?
- Trouver un état dans lequel l'heuristique de Gaschnig retourne une valeur strictement plus grande que misplaced et manhattan. 
Maintenant, il faut la coder.

In [ ]:
%%add_to SBState
def gaschnig(self):
    #TODO

In [ ]:
%%add_to SlidingBlock         
def gaschnig(self, state):
    return state.gaschnig()

Évaluons désormais nos trois heuristiques.

In [ ]:
eval_mis = []
eval_man = []
eval_gas = []
ns = range(3,9)
for n in ns :
    eval_mis.append(evaluate(n,n*3,40, sb.misplaced))
    eval_man.append(evaluate(n,n*3,40, sb.manhattan))
    eval_gas.append(evaluate(n,n*3,40, sb.gaschnig))

import matplotlib.pyplot as plt
plt.plot(ns, [x for (x,_) in eval_mis], 'r--', ns, [x for (x,_) in eval_man], 'bs', ns, [x for (x,_) in eval_gas], 'g^')
plt.xlabel('size of puzzle')
plt.ylabel('Times (s)')
plt.show()

plt.plot(ns, [x for (_,x) in eval_mis], 'r--', ns, [x for (_,x) in eval_man], 'bs', ns, [x for (_,x) in eval_gas], 'g^')
plt.xlabel('size of puzzle')
plt.ylabel('#Nodes extended')
plt.show()

On considère un taquin de taille $4 \times 4$ avec la configuration suivante :

| | | | |
|-----|-----|-----|-----|
| '5' | '10' | '7' | '9' |
| '8' | '4' | '6' | '2'|
| '1' | '11' | '3' | '13'|
| '12' | '14' | '15' | 'e' |

On se propose de découper le problème du taquin en quatre sous-problèmes. Le premier consiste à ne s'intéresser qu'aux tuiles 1, 2, 3 et 4. On cherche alors à passer de la configuration représenter à gauche ci-dessous 

| | | | |
|-----|-----|-----|-----|
| X | X | X | X |
| X | '4' | X | '2'|
| '1' | X | '3' | X|
| X | X | X | 'e' |

à celle-ci sans se soucier des autres tuiles.

| | | | |
|-----|-----|-----|-----|
| '1' | '2' | '3' | '4' |
| X | X | X | X |
| X | X | X | X |
| X | X | X | 'e' |

Soit $h_1$ le nombre de pas pour passer de la première configuration à la seconde avec les règles habituelles du taquin  **en ne comptant que les déplacement des pièces numérotées de 1 à 4**. 
On définit de la même manière les valeurs: 
- $h_2$ pour les tuiles 5,6,7,8,  
- $h_3$ pour les tuiles 9,10,11,12,
- et $h_4$ pour les tuiles 13,14,15. 

Voici quelques questions:
- la valeur $h = h_1 + h_2 + h_3 + h 4$ est elle une heuristique admissible (justifiez votre réponse) ? 
- Si oui, que peut on dire de cette heursitque par rapport à l'heuristique utilisant la distance de Manhattan ?